In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from keybert import KeyBERT
import time
from tqdm.notebook import tqdm
from spacy import displacy

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
sds_text = pd.read_csv('../data/sds_text.csv')

In [ ]:
sds_text = sds_text.set_index('Unnamed: 0')

In [ ]:
nlp.components

## Generating tokens

In [ ]:
def preprocess_text(text):
    doc = nlp(text)
    tokens = [token.lemma_.lower().strip() for token in doc if not token.is_stop
             and not token.is_digit
             and not token.is_punct
             and not token.is_space]
    return tokens

In [ ]:

sds_text['tokens'] = sds_text['episode_split_text'].apply(preprocess_text)

In [ ]:
sds_text

## Entity Detection

In [ ]:
def entity_detection(text):
    doc = nlp(text)
    entity = [(i, i.label_, i.label) for i in doc.ents]
    #entity = displacy.render(doc, style = 'ent')
    return entity

In [ ]:
sds_1['entity'] = sds_1['episode_split_text'].apply(entity_detection)

In [ ]:
sds_1

In [ ]:
sds_text['entity'] = sds_text['episode_split_text'].apply(entity_detection)

In [ ]:
doc

In [ ]:
displacy.render(doc, style = 'ent')

## Sentiment Analysis

In [ ]:
from spacytextblob.spacytextblob import SpacyTextBlob

In [ ]:
nlp.add_pipe("spacytextblob")

In [ ]:
def sentiment_analysis(text):
    doc = nlp(text)
    sentiment = doc._.blob.polarity
    return sentiment

In [ ]:
sds_1 = sds_text[sds_text['episode_number'] == 1]

In [ ]:
sds_1

In [ ]:
sds_1['sentiments'] = sds_1['episode_split_text'].apply(sentiment_analysis)

In [ ]:
sds_1

In [ ]:
nlp.components

In [ ]:
sds_text['sentiments'] = sds_text['episode_split_text'].apply(sentiment_analysis)

In [ ]:
sds_text['tokens'].loc[0]

In [ ]:
mytext = sds_text['tokens'].loc[0]

In [ ]:
docx = nlp(mytext)

In [ ]:
#check for sentiment polarity
docx._.polarity

In [ ]:
# check for subjectivity
docx._.subjectivity

In [ ]:
#Check Assessment:list polarity/subject for assessed token
docx._.assessments

In [ ]:
sds_text

In [ ]:
sds_text.info()

In [ ]:
sds_processed = pd.DataFrame(sds_text.groupby('episode_number')['episode_split_text'].agg(lambda x: ' '.join(x)))

In [ ]:
sds_processed

In [ ]:
sds_processed['episode_split_text'].loc[683]

In [ ]:
def keyword_extractor(token):
    doc = token
    kw_model = KeyBERT()
    keyword = ''
    stopwords = list(STOP_WORDS)
    for keyword in doc:
        keywords = kw_model.extract_keywords(doc,keyphrase_ngram_range=(1, 1), stop_words=stopwords)
    return keywords

In [ ]:
sds_processed['keywords'] = sds_processed['episode_split_text'].apply(keyword_extractor)

In [ ]:
keywords

In [ ]:
stopwords = list(STOP_WORDS)

In [ ]:
# https://maartengr.github.io/BERTopic/index.html